## Wake County Analysis 2000 - 2017
Author: Emily Padvorac

December 2018

__Wake County__ in __North Carolina__ has experienced significant growth since 2000.  This analysis aims to observe and assess changes in Wake County, North Carolina from 2000 - 2017 to asses the changes that occured in the demographics, and housing market.  This study uses census data from American Fact Finder tool.  

Possible implications for this analysis could be used for zoning purposes, transportation, etc.

__[API Census Reference Guide](https://www.census.gov/content/dam/Census/data/developers/api-user-guide/api-guide.pdf)__

__[List of Census Data available with API](https://api.census.gov/data.html)__

__[Get your API key from here](https://api.census.gov/data/key_signup.html)__

__Recommended:__ In order to keep your API key confidential, please save your API key in a .py file named censusAPI.py as: __myAPI = 'XXXXXXXXXXXXXXX'__ and read into this notebook as: __from censusAPI import myAPI__

In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pylab as pl
import matplotlib
import matplotlib.pyplot as plt
import pylab as pl
import pysal
import seaborn as sns
sns.set_style('whitegrid')

%pylab inline

from censusAPI import myAPI

Populating the interactive namespace from numpy and matplotlib


# 1. Data

# 1.1 Population and Demographics
## Census Bureau 2000 Decennial: Summary File 3 + Population Estimates 2016
Source: U.S. Census Bureau Decennial Census 2010; U.S. Census Bureau Population Estimates Program 2016


## Obtaining data for 2010, from the Summary File 3 survey, for Wake County (183) and NC State (37)

Variables to be obtained:

- Total population
- Race
- Employment
- Median Income
- Housing be tenure (% renters)
- Median Home Value and Median Rent
- % Collage
- Age Groups?
- urban vs. rural?


__[variables](https://api.census.gov/data/2000/sf1/variables.html)__

__[examples for reading in with API](https://api.census.gov/data/2000/sf1/examples.html)__

In [3]:
wc00 = pd.read_json('https://api.census.gov/data/2000/sf3?get=P001001,'+
                     'H001001,H007002,H007003,H015002,H015003,'+
                     'H062001,H069010,H076001,P005002,P005005,'+
                     'P006002,P006003,P006005,P006008,P007010,P019001,'+
                     'P019002,P019024,P019046,P052002,P052003,'+
                     'P052004,P052005,P052006,P052007,P052008,'+
                     'P052009,P052010,P052011,P052012,P052013,'+
                     'P052014,P052015,P052016,P052017,P037032,'+
                     'P037015,PCT045001,'+
                     'NAME&for=county:183&in=state:37&key='+myAPI)
wc00

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,P001001,H001001,H007002,H007003,H015002,H015003,H062001,H069010,H076001,P005002,...,P052014,P052015,P052016,P052017,P037032,P037015,PCT045001,NAME,state,county
1,627846,258953,159456,82584,425545,183108,82130,12942,162900,553167,...,20325,10980,9040,7178,60512,58877,35482,Wake County,37,183


In [4]:
wc00 = wc00.rename(columns=wc00.iloc[0])
wc00 = wc00.iloc[1:]

In [5]:
wc00.columns = ['TotalPop','TotalHU', 'HU_Owned', 'HU_Rented', 'Pop_HU_Owned',
               'Pop_HU_rented', 'Rent', 'rentBurden50%','H_Value', 'Urban',
               'Rural', 'White','Black','Asian', '2+_Races', 'HispanicLatino','5+','5-17',
                '18-64','65+','inc_<10','inc_10-15','inc_15-20','inc_20-25',
               'inc_25-30','inc_30-35','inc_35-40','inc_40-45','inc_45-50',
               'inc_50-60','inc_60-75','inc_75-100','inc_100-125','inc_125-150',
               'inc_150-200','inc_200+','bachelorFemale','bachelorMale','medInc','Name',
               'State','County']

In [6]:
wc00 = wc00.set_index(['Name'])

In [7]:
wc00

,TotalPop,TotalHU,HU_Owned,HU_Rented,Pop_HU_Owned,Pop_HU_rented,Rent,rentBurden50%,H_Value,Urban,...,inc_75-100,inc_100-125,inc_125-150,inc_150-200,inc_200+,bachelorFemale,bachelorMale,medInc,State,County
Name,,,,,,,,,,,,,,,,,,,,,
Wake County,627846,258953,159456,82584,425545,183108,82130,12942,162900,553167,...,33970,20325,10980,9040,7178,60512,58877,35482,37,183


In [8]:
nc00 = pd.read_json('https://api.census.gov/data/2000/sf3?get=P001001,'+
                     'H001001,H007002,H007003,H015002,H015003,'+
                     'H062001,H069010,H076001,P005002,P005005,'+
                     'P006002,P006003,P006005,P006008,P007010,P019001,'+
                     'P019002,P019024,P019046,P052002,P052003,'+
                     'P052004,P052005,P052006,P052007,P052008,'+
                     'P052009,P052010,P052011,P052012,P052013,'+
                     'P052014,P052015,P052016,P052017,P037032,'+
                     'P037015,PCT045001,'+
                     'NAME&for=state:37&key='+myAPI)

nc00 = nc00.rename(columns=nc00.iloc[0])
nc00 = nc00.iloc[1:]

In [9]:
nc00.columns = ['TotalPop','TotalHU', 'HU_Owned', 'HU_Rented', 'Pop_HU_Owned',
               'Pop_HU_rented', 'Rent', 'rentBurden50%','H_Value', 'Urban',
               'Rural', 'White','Black','Asian', '2+_Races', 'HispanicLatino','5+','5-17',
                '18-64','65+','inc_<10','inc_10-15','inc_15-20','inc_20-25',
               'inc_25-30','inc_30-35','inc_35-40','inc_40-45','inc_45-50',
               'inc_50-60','inc_60-75','inc_75-100','inc_100-125','inc_125-150',
               'inc_150-200','inc_200+','bachelorFemale','bachelorMale','medInc','Name',
               'State']

nc00 = nc00.set_index(['Name'])
nc00

,TotalPop,TotalHU,HU_Owned,HU_Rented,Pop_HU_Owned,Pop_HU_rented,Rent,rentBurden50%,H_Value,Urban,...,inc_60-75,inc_75-100,inc_100-125,inc_125-150,inc_150-200,inc_200+,bachelorFemale,bachelorMale,medInc,State
Name,,,,,,,,,,,,,,,,,,,,,
North Carolina,8049313,3523944,2172270,959743,5553133,2242172,944275,148859,108300,4847075,...,314899,279020,128591,60030,50650,55604,422749,385321,26173,37


In [10]:
pop00 = wc00.T.merge(nc00.T, how='left', left_index = True, right_index = True).T
print(pop00.shape)
pop00

(2, 41)


,TotalPop,TotalHU,HU_Owned,HU_Rented,Pop_HU_Owned,Pop_HU_rented,Rent,rentBurden50%,H_Value,Urban,...,inc_75-100,inc_100-125,inc_125-150,inc_150-200,inc_200+,bachelorFemale,bachelorMale,medInc,State,County
Name,,,,,,,,,,,,,,,,,,,,,
Wake County,627846,258953,159456,82584,425545,183108,82130,12942,162900,553167,...,33970,20325,10980,9040,7178,60512,58877,35482,37,183
North Carolina,8049313,3523944,2172270,959743,5553133,2242172,944275,148859,108300,4847075,...,279020,128591,60030,50650,55604,422749,385321,26173,37,NaN


In [11]:
pop00['Degree'] = pop00['bachelorFemale'] + pop00['bachelorMale']
pop00.drop(['bachelorFemale','bachelorMale'], axis=1, inplace=True)
pop00 = pop00.fillna(0)
pop00 = pop00.astype(int)

In [12]:
pop00

,TotalPop,TotalHU,HU_Owned,HU_Rented,Pop_HU_Owned,Pop_HU_rented,Rent,rentBurden50%,H_Value,Urban,...,inc_60-75,inc_75-100,inc_100-125,inc_125-150,inc_150-200,inc_200+,medInc,State,County,Degree
Name,,,,,,,,,,,,,,,,,,,,,
Wake County,627846,258953,159456,82584,425545,183108,82130,12942,162900,553167,...,29279,33970,20325,10980,9040,7178,35482,37,183,6051258877
North Carolina,8049313,3523944,2172270,959743,5553133,2242172,944275,148859,108300,4847075,...,314899,279020,128591,60030,50650,55604,26173,37,0,422749385321


## ACS 1-year  supplemental estimates 2016

Obtained for state and county level

__[variables](https://api.census.gov/data/2016/acs/acsse/variables.html)__

__[examples](https://api.census.gov/data/2016/acs/acsse/examples.html)__


In [13]:
wc16 = pd.read_json('https://api.census.gov/data/2016/acs/acsse?get=K200101_001E,'+
                     'K202501_002E,K202502_002E,K202502_003E,K202503_002E,K202503_003E,'+
                    'K202511_001E,K202510_001E,K200201_002E,K200201_003E,K200201_005E,'+
                    'K200201_008E,K200301_003E,K200104_002E,K200104_003E,K200104_004E,'+
                    'K200104_005E,K200104_006E,K200104_007E,K200104_008E,K201901_002E,'+
                    'K201901_003E,K201901_004E,K201901_005E,K201901_006E,K201901_007E,'+
                    'K201901_008E,K201501_007E,K201902_001E,'+
                     'NAME&for=county:183&in=state:37&key='+myAPI)

In [14]:
wc16 = wc16.rename(columns=wc16.iloc[0])
wc16 = wc16.iloc[1:]
print(wc16.shape)
wc16

(1, 32)


,K200101_001E,K202501_002E,K202502_002E,K202502_003E,K202503_002E,K202503_003E,K202511_001E,K202510_001E,K200201_002E,K200201_003E,...,K201901_004E,K201901_005E,K201901_006E,K201901_007E,K201901_008E,K201501_007E,K201902_001E,NAME,state,county
1,1046791,387771,242287,145484,672763,352082,1063,268400,688226,213306,...,60853,94890,67243,36437,36294,233457,76097,"Wake County, North Carolina",37,183


In [15]:
wc16.columns = ['TotalPop','TotalHU', 'HU_Owned', 'HU_Rented', 'Pop_HU_Owned',
               'Pop_HU_rented', 'Rent', 'H_Value', 'White', 'Black', 'Asian','2+',
               'HispanicLatino','<18','18-24','25-34','35-44','45-54','55-64',
               '65+','inc_<20','inc_20-40','inc_40-60','inc_60-100','inc_100-150',
               'inc_150-200','inc_200+','Bachelor','medInc','Name','State','County']

wc16 = wc16.set_index(['Name'])
wc16 = wc16.rename({'Wake County, North Carolina': 'Wake County'})
wc16

,TotalPop,TotalHU,HU_Owned,HU_Rented,Pop_HU_Owned,Pop_HU_rented,Rent,H_Value,White,Black,...,inc_20-40,inc_40-60,inc_60-100,inc_100-150,inc_150-200,inc_200+,Bachelor,medInc,State,County
Name,,,,,,,,,,,,,,,,,,,,,
Wake County,1046791,387771,242287,145484,672763,352082,1063,268400,688226,213306,...,56059,60853,94890,67243,36437,36294,233457,76097,37,183


In [16]:
nc16 = pd.read_json('https://api.census.gov/data/2016/acs/acsse?get=K200101_001E,'+
                     'K202501_002E,K202502_002E,K202502_003E,K202503_002E,K202503_003E,'+
                    'K202511_001E,K202510_001E,K200201_002E,K200201_003E,K200201_005E,'+
                    'K200201_008E,K200301_003E,K200104_002E,K200104_003E,K200104_004E,'+
                    'K200104_005E,K200104_006E,K200104_007E,K200104_008E,K201901_002E,'+
                    'K201901_003E,K201901_004E,K201901_005E,K201901_006E,K201901_007E,'+
                    'K201901_008E,K201501_007E,K201902_001E,'+
                     'NAME&for=state:37&key='+myAPI)
nc16 = nc16.rename(columns=nc16.iloc[0])
nc16 = nc16.iloc[1:]
print(nc16.shape)
nc16

(1, 31)


,K200101_001E,K202501_002E,K202502_002E,K202502_003E,K202503_002E,K202503_003E,K202511_001E,K202510_001E,K200201_002E,K200201_003E,...,K201901_003E,K201901_004E,K201901_005E,K201901_006E,K201901_007E,K201901_008E,K201501_007E,K201902_001E,NAME,state
1,10146788,3882423,2493388,1389035,6444693,3446133,839,165400,6989065,2182212,...,825286,688144,863182,449215,166800,169392,1345615,50584,North Carolina,37


In [17]:
nc16.columns = ['TotalPop','TotalHU', 'HU_Owned', 'HU_Rented', 'Pop_HU_Owned',
               'Pop_HU_rented', 'Rent', 'H_Value', 'White', 'Black', 'Asian','2+',
               'HispanicLatino','<18','18-24','25-34','35-44','45-54','55-64',
               '65+','inc_<20','inc_20-40','inc_40-60','inc_60-100','inc_100-150',
               'inc_150-200','inc_200+','Bachelor','medInc','Name','State']

nc16 = nc16.set_index(['Name'])

In [18]:
pop16 = wc16.T.merge(nc16.T, how='left', left_index = True, right_index = True).T
print(pop16.shape)
pop16

(2, 31)


,TotalPop,TotalHU,HU_Owned,HU_Rented,Pop_HU_Owned,Pop_HU_rented,Rent,H_Value,White,Black,...,inc_20-40,inc_40-60,inc_60-100,inc_100-150,inc_150-200,inc_200+,Bachelor,medInc,State,County
Name,,,,,,,,,,,,,,,,,,,,,
Wake County,1046791,387771,242287,145484,672763,352082,1063,268400,688226,213306,...,56059,60853,94890,67243,36437,36294,233457,76097,37,183
North Carolina,10146788,3882423,2493388,1389035,6444693,3446133,839,165400,6989065,2182212,...,825286,688144,863182,449215,166800,169392,1345615,50584,37,NaN


In [19]:
pop16 = pop16.fillna(0)
pop16 = pop16.astype(int)

# 1.2 Housing
## Census Bureau Building Permits Survey 
Source: U.S. Census Bureau Building Permit Survey

__[U.S. Census Bureau Building Permit Survey](https://www.census.gov/construction/bps/)__

Data was downloaded for Permits by County then converted to .csv and uploaded for Wake County, NC only for the years 2000, 2010, 2016.

__[County Data](https://www2.census.gov/econ/bps/)__


__[State Data](https://www.census.gov/construction/bps/stateannual.html)__


In [22]:
tablesCO = ['co2016a','co2015a', 'co2014a','co2013a', 'co2012a', 
            'co2011a', 'co2010a','co2009a', 'co2008a',
            'co2007a', 'co2006a','co2005a', 'co2004a',
            'co2003a', 'co2002a','co2001a', 'co2000a']

Wakedata = []

for year in tablesCO:
    df = 'co'+year

    df = pd.read_table('https://www2.census.gov/econ/bps/County/co2016a.txt', 
            header=0, sep=r'\,|\t', engine='python').iloc[:,:18]

    df.columns = df.iloc[0]
    df = df[1:].set_index(['Name'])

    df = df.drop(['Code','Bldgs', 'Value'], axis=1)
    df.columns = ['State', 'County', '1unit', '2unit', '3-4unit', '5+unit']
    
    df = df[df['State'] == '37']
    df = df[df['County'] =='183']
    
    df = df.astype(int)
    df['1-2units'] = df['1unit'] + df['2unit']
    df['3+units'] = df['3-4unit'] + df['5+unit']
    df = df.drop(['1unit', '2unit', '3-4unit', '5+unit'], axis=1)

    df['stco'] = df['State'].astype(str) + df['County'].astype(str)
    df = df.drop(['State','County'], axis = 1)
    df = df.astype(int)
    Wakedata.append(df)

Wakeall = pd.concat(Wakedata).groupby('stco').sum()
print(Wakeall.shape)
print(Wakeall.dtypes)
Wakeall.head()

(1, 2)
1-2units    int64
3+units     int64
dtype: object


,1-2units,3+units
stco,,
37183,119119,68731


In [21]:
tablesCO = ['co2016a','co2015a', 'co2014a','co2013a', 'co2012a', 
            'co2011a', 'co2010a','co2009a', 'co2008a',
            'co2007a', 'co2006a','co2005a', 'co2004a',
            'co2003a', 'co2002a','co2001a', 'co2000a']

NCdata = []

for year in tablesCO:
    df = 'co'+year

    df = pd.read_table('https://www2.census.gov/econ/bps/County/co2016a.txt', 
            header=0, sep=r'\,|\t', engine='python').iloc[:,:18]

    df.columns = df.iloc[0]
    df = df[1:].set_index(['Name'])

    df = df.drop(['Code','Bldgs', 'Value'], axis=1)
    df.columns = ['State', 'County', '1unit', '2unit', '3-4unit', '+5unit']
    
    df = df[df['State'] == '37']
    
    df = df.astype(int)
    df['1-2units'] = df['1unit'] + df['2unit']
    df['+3units'] = df['3-4unit'] + df['+5unit']
    df = df.drop(['1unit', '2unit', '3-4unit', '+5unit'], axis=1)

    df['stco'] = df['State'].astype(str) + df['County'].astype(str)
    df = df.drop(['State','County'], axis = 1)
    df = df.astype(int)
    NCdata.append(df)

NCpermits = pd.concat(NCdata).groupby('stco').sum()
print(NCpermits.shape)
print(NCpermits.dtypes)
NCpermits.head()

(100, 2)
1-2units    int64
+3units     int64
dtype: object


,1-2units,+3units
stco,,
371,16150,6851
373,731,0
375,493,0
377,697,0
379,1428,0


# 1.3 Employment

__[U.S. Bureau of Labor Statistics](https://www.bls.gov/cew/datatoc.htm)__

Source: U.S. Bureau of Labor Statistics Quarterly Census of Employment and Wage(QCEW) North American Industry Classification System (NAICS)

Data was downloaded for all counties in the state of North Carolina, and in Wake County for the years 2000 and 2016.  The change in employment will be calculated for the state of North Carolina and Wake County.


In [23]:
wakeJobs00 = pd.read_csv('2000.annual 37183 Wake County, North Carolina.csv')

wakeJobs16 = pd.read_csv('2016.annual 37183 Wake County, North Carolina.csv')

print(wakeJobs00.shape)
print(wakeJobs16.shape)

wakeJobs16.columns

(1059, 43)
(1950, 43)


Index(['area_fips', 'own_code', 'industry_code', 'agglvl_code', 'size_code',
       'year', 'qtr', 'disclosure_code', 'area_title', 'own_title',
       'industry_title', 'agglvl_title', 'size_title',
       'annual_avg_estabs_count', 'annual_avg_emplvl', 'total_annual_wages',
       'taxable_annual_wages', 'annual_contributions', 'annual_avg_wkly_wage',
       'avg_annual_pay', 'lq_disclosure_code', 'lq_annual_avg_estabs_count',
       'lq_annual_avg_emplvl', 'lq_total_annual_wages',
       'lq_taxable_annual_wages', 'lq_annual_contributions',
       'lq_annual_avg_wkly_wage', 'lq_avg_annual_pay', 'oty_disclosure_code',
       'oty_annual_avg_estabs_count_chg',
       'oty_annual_avg_estabs_count_pct_chg', 'oty_annual_avg_emplvl_chg',
       'oty_annual_avg_emplvl_pct_chg', 'oty_total_annual_wages_chg',
       'oty_total_annual_wages_pct_chg', 'oty_taxable_annual_wages_chg',
       'oty_taxable_annual_wages_pct_chg', 'oty_annual_contributions_chg',
       'oty_annual_contributions_pct_

In [24]:
wakeJobs16 = pd.DataFrame(wakeJobs16.T.loc['annual_avg_emplvl'])[:1]
wakeJobs16

,annual_avg_emplvl
0,530547


In [25]:
wakeJobs00 = pd.DataFrame(wakeJobs00.T.loc['annual_avg_emplvl'])[:1]
wakeJobs00

,annual_avg_emplvl
0,382702


In [26]:
wakeJobs = pd.concat([wakeJobs00, wakeJobs16])
wakeJobs.columns = ['WakeCo']
wakeJobs = wakeJobs.T
wakeJobs.columns = ['2000', '2016']
wakeJobs

,2000,2016
WakeCo,382702,530547


In [27]:
wakeJobs['change'] = wakeJobs['2016'] - wakeJobs['2000']
wakeJobs['%change'] = ((wakeJobs['2016'] - wakeJobs['2000']) / wakeJobs['2000']) *100
wakeJobs

,2000,2016,change,%change
WakeCo,382702,530547,147845,38.6319


In [28]:
ncJobs00 = pd.read_csv('2000.annual 37000 North Carolina -- Statewide.csv')

ncJobs16 = pd.read_csv('2016.annual 37000 North Carolina -- Statewide.csv')

print(ncJobs00.shape)
print(ncJobs16.shape)

ncJobs16.columns

(2230, 43)
(2747, 43)


Index(['area_fips', 'own_code', 'industry_code', 'agglvl_code', 'size_code',
       'year', 'qtr', 'disclosure_code', 'area_title', 'own_title',
       'industry_title', 'agglvl_title', 'size_title',
       'annual_avg_estabs_count', 'annual_avg_emplvl', 'total_annual_wages',
       'taxable_annual_wages', 'annual_contributions', 'annual_avg_wkly_wage',
       'avg_annual_pay', 'lq_disclosure_code', 'lq_annual_avg_estabs_count',
       'lq_annual_avg_emplvl', 'lq_total_annual_wages',
       'lq_taxable_annual_wages', 'lq_annual_contributions',
       'lq_annual_avg_wkly_wage', 'lq_avg_annual_pay', 'oty_disclosure_code',
       'oty_annual_avg_estabs_count_chg',
       'oty_annual_avg_estabs_count_pct_chg', 'oty_annual_avg_emplvl_chg',
       'oty_annual_avg_emplvl_pct_chg', 'oty_total_annual_wages_chg',
       'oty_total_annual_wages_pct_chg', 'oty_taxable_annual_wages_chg',
       'oty_taxable_annual_wages_pct_chg', 'oty_annual_contributions_chg',
       'oty_annual_contributions_pct_

In [29]:
ncJobs16 = pd.DataFrame(ncJobs16.T.loc['annual_avg_emplvl'])[:1]
ncJobs00 = pd.DataFrame(ncJobs00.T.loc['annual_avg_emplvl'])[:1]

In [30]:
ncJobs = pd.concat([ncJobs00, ncJobs16])
ncJobs.columns = ['NC']
ncJobs = ncJobs.T
ncJobs.columns = ['2000', '2016']
ncJobs

,2000,2016
NC,3856286,4259276


In [31]:
ncJobs['change'] = ncJobs['2016'] - ncJobs['2000']
ncJobs['%change'] = ((ncJobs['2016'] - ncJobs['2000']) / ncJobs['2000']) *100
ncJobs

,2000,2016,change,%change
NC,3856286,4259276,402990,10.4502


In [32]:
jobs = pd.concat([wakeJobs, ncJobs])
jobs

,2000,2016,change,%change
WakeCo,382702,530547,147845,38.6319
NC,3856286,4259276,402990,10.4502
